In [ ]:
#!pip install ragas

In [ ]:
#!pip install langchain-openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<<OPEN_API_PERSONAL_KEY_HERE>>"

In [ ]:
from langchain.docstore.document import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import time

In [ ]:
CHROMA_PERSIST_DIRECTORY = "./chroma_db"
EMBEDDINGS_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
occupation = "politician"
target_collection_name_2 = occupation + "_template_files" 
embeddings_2 = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
print("Target Collection Is: " + target_collection_name_2)
chroma_db_from_disk_2 = Chroma(persist_directory=CHROMA_PERSIST_DIRECTORY,                             
                            collection_name=target_collection_name_2,
                            embedding_function=embeddings_2)
results_templates = chroma_db_from_disk_2._collection.get(include=["documents", "metadatas", "embeddings"])
documents_templates = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(results_templates["documents"], results_templates["metadatas"])
]
template_sentences = results_templates["documents"][2] ## Can use any here
template_sentences

In [ ]:
type(template_sentences)

In [ ]:
occupation = "politician"
CHROMA_CHUNK_SIZE = 1000
CHROMA_PERSIST_DIRECTORY = "./chroma_db"
CHROMA_COLLECTION_NAME = occupation + "_generated_articles"
EMBEDDINGS_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
start_time = time.time()
target_collection_name = occupation + "_generated_articles"
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
print("Target Collection Is: " + target_collection_name)
chroma_db_from_disk = Chroma(persist_directory=CHROMA_PERSIST_DIRECTORY,                             
                            collection_name=target_collection_name,
                            embedding_function=embeddings)
results = chroma_db_from_disk._collection.get(include=["documents", "metadatas", "embeddings"])
documents = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(results["documents"], results["metadatas"])
]
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")
print("Number of Documents: "+  str(len(documents)))

In [ ]:
import ast
import json
from ragas import SingleTurnSample 
from ragas.metrics import Faithfulness, ResponseRelevancy
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4"))
evaluator_embeddings = OpenAIEmbeddings()
count = 0 
with open("Ragas_Evaluation_V2.json","w") as f:
    for doc in documents:
        count += 1
        #rint(doc)
        page_content = doc.page_content
        my_metadata = doc.metadata
        #rint(page_content)
        first_name = my_metadata["first_name"]
        last_name = my_metadata["last_name"]
        entity_name = first_name + " " +last_name
        response_marker = "### Response:"
        ########## Parse Generated Artcicle ###########
        try:
            if response_marker in page_content:
                response_start = page_content.index(response_marker) + len(response_marker)
                generated_article = page_content[response_start:].strip() 
                #rint("Extracted Response:")
                #rint(generated_article)
        except (SyntaxError, ValueError) as e:
            print(f"Error parsing JSON: {e}")
            continue
        ########## Get JSON from Chroma  ##########
        target_collection_name = occupation + "_json_files" 
        embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
        #rint("Target Collection Is: " + target_collection_name)
        chroma_db_from_disk = Chroma(persist_directory=CHROMA_PERSIST_DIRECTORY,                             
                                    collection_name=target_collection_name,
                                    embedding_function=embeddings)
        results = chroma_db_from_disk.get(
            where={
                "$and": [
                    {"first_name": first_name},
                    {"last_name": last_name}
                ]
            },
            include=["documents", "metadatas"])
        documents = [
            Document(page_content=doc, metadata=meta)
            for doc, meta in zip(results["documents"], results["metadatas"])
        ]
        document = documents[0]
        content = ast.literal_eval(document.page_content)[0]
        response_start = content.index("### Response:")
        json_start = content.index("{", response_start)
        json_end = content.rindex("}")
        json_string = content[json_start:json_end+1]
        try:
            parsed_dict = ast.literal_eval(json_string)
            json_data = json.dumps(parsed_dict, ensure_ascii=False, indent=2)
            #print(json_data)
        except (SyntaxError, ValueError) as e:
            print(f"Error parsing JSON: {e}")
            continue
        ########### Faithfulness ##############
        prompt = f"Generate an Article in Hindi Language for {entity_name} using the details given in the json string below. Strictly use the given template sentences as a reference. Do not any other extra English text after the article."
        sample = SingleTurnSample(
        user_input= prompt, 
        response=generated_article,
        retrieved_contexts=[
            #template_sentences,
            json_data
                ]
            )
        scorer = Faithfulness()
        scorer.llm = evaluator_llm
        faithfulness_score = await scorer.single_turn_ascore(sample)
        print("FaithFulness Score Is: " + str(faithfulness_score))
        ########### Relevancy ###################
        rel_scorer = ResponseRelevancy()
        rel_scorer.llm = evaluator_llm
        rel_scorer.embeddings = evaluator_embeddings
        relevancy_score = await rel_scorer.single_turn_ascore(sample)
        print("Relevancy score is: " + str(relevancy_score))
        op = { "First Name": first_name, "Last Name": last_name, "Faithfullness Score": faithfulness_score, 
              "Relevancy": relevancy_score}
        json.dump(op, f, indent=4)
        print("count" + str(count))